**Compute Key-Performance Indicators (KPIs) from the MLflow server.**

# IMPORTS

In [1]:
import mlflow
import plotly.express as px
import polars as pl

# OPTIONS

In [2]:
pl.Config.set_tbl_cols(n=-1)

polars.config.Config

# CONFIGS

In [3]:
MAX_RESULTS = 100
TRACKING_URI = "http://localhost:5000"
REGISTRY_URI = "http://localhost:5000"

# CLIENTS

In [4]:
client = mlflow.tracking.MlflowClient(tracking_uri=TRACKING_URI, registry_uri=REGISTRY_URI)

# INDICATORS

In [5]:
experiments = client.search_experiments(
    view_type=mlflow.entities.ViewType.ALL, max_results=MAX_RESULTS, order_by=["creation_time DESC"]
)
experiments = [dict(experiment) for experiment in experiments]
experiments = (
    pl.DataFrame(experiments)
    .with_columns(
        pl.from_epoch("creation_time", time_unit="ms"),
        pl.from_epoch("last_update_time", time_unit="ms"),
    )
    .drop("tags")
)
experiments

artifact_location,creation_time,experiment_id,last_update_time,lifecycle_stage,name
str,datetime[ms],str,datetime[ms],str,str
"""file:///home/fmind/mlops-pytho…",2024-12-09 19:31:02.967,"""892046783494697990""",2024-12-09 19:31:02.967,"""active""","""bikes"""


In [6]:
runs = client.search_runs(
    experiment_ids=experiments["experiment_id"].unique(),
    run_view_type=mlflow.entities.ViewType.ALL,
    max_results=MAX_RESULTS,
    order_by=["created DESC"],
)
runs = [dict(run.info) | dict(run.data) for run in runs]
runs = (
    pl.DataFrame(runs)
    .with_columns(
        pl.from_epoch("start_time", time_unit="ms"),
        pl.from_epoch("end_time", time_unit="ms"),
        (pl.col("end_time") - pl.col("start_time")).cast(pl.Int64).alias("run_time"),
    )
    .drop("metrics", "params", "tags")
)
runs

artifact_uri,end_time,experiment_id,lifecycle_stage,run_id,run_name,run_uuid,start_time,status,user_id,run_time
str,datetime[ms],str,str,str,str,str,datetime[ms],str,str,i64
"""file:///home/fmind/mlops-pytho…",2024-12-09 21:26:50.094,"""892046783494697990""","""active""","""c21b823a68ba42b2b1ff98b2ae73c4…","""Explanations""","""c21b823a68ba42b2b1ff98b2ae73c4…",2024-12-09 21:26:19.324,"""FINISHED""","""fmind""",30770
"""file:///home/fmind/mlops-pytho…",2024-12-09 21:26:18.061,"""892046783494697990""","""active""","""9da0f31e36a9413eaab18149ace82c…","""Evaluations""","""9da0f31e36a9413eaab18149ace82c…",2024-12-09 21:26:13.489,"""FINISHED""","""fmind""",4572
"""file:///home/fmind/mlops-pytho…",2024-12-09 21:26:12.215,"""892046783494697990""","""active""","""8316d6855d17401b8358e88a48c159…","""Inference""","""8316d6855d17401b8358e88a48c159…",2024-12-09 21:26:08.210,"""FINISHED""","""fmind""",4005
"""file:///home/fmind/mlops-pytho…",2024-12-09 21:26:06.933,"""892046783494697990""","""active""","""b92f959ab1564eb897e2f66f7089f5…","""Promotion""","""b92f959ab1564eb897e2f66f7089f5…",2024-12-09 21:26:03.382,"""FINISHED""","""fmind""",3551
"""file:///home/fmind/mlops-pytho…",2024-12-09 21:26:01.071,"""892046783494697990""","""active""","""fc31f5b645204fdd9ef58fbbc0c105…","""Training""","""fc31f5b645204fdd9ef58fbbc0c105…",2024-12-09 21:25:39.001,"""FINISHED""","""fmind""",22070
…,…,…,…,…,…,…,…,…,…,…
"""file:///home/fmind/mlops-pytho…",2024-12-09 19:33:10.574,"""892046783494697990""","""active""","""ca4c7550ad354be9a2f10f7eade8d4…","""Training""","""ca4c7550ad354be9a2f10f7eade8d4…",2024-12-09 19:32:47.406,"""FINISHED""","""fmind""",23168
"""file:///home/fmind/mlops-pytho…",2024-12-09 19:32:45.039,"""892046783494697990""","""active""","""3c3f2883be0643b9800e8c161764bb…","""resilient-doe-203""","""3c3f2883be0643b9800e8c161764bb…",2024-12-09 19:31:03.107,"""FINISHED""","""fmind""",101932
"""file:///home/fmind/mlops-pytho…",2024-12-09 19:32:45.039,"""892046783494697990""","""active""","""846fb4b5ca654800b2cb0592c6c2e9…","""unequaled-conch-92""","""846fb4b5ca654800b2cb0592c6c2e9…",2024-12-09 19:31:03.107,"""FINISHED""","""fmind""",101932


In [7]:
models_ = client.search_registered_models(
    max_results=MAX_RESULTS, order_by=["creation_timestamp DESC"]
)
models = []
for model in models_:
    model = dict(model)
    model.pop("latest_versions")
    models.append(model)
models = (
    pl.DataFrame(models)
    .with_columns(
        pl.from_epoch("creation_timestamp", time_unit="ms"),
        pl.from_epoch("last_updated_timestamp", time_unit="ms"),
    )
    .drop("aliases", "tags")
)
models

creation_timestamp,description,last_updated_timestamp,name
datetime[ms],str,datetime[ms],str
2024-12-09 19:33:09.416,"""""",2024-12-09 21:26:06.439,"""bikes"""


In [8]:
versions_ = client.search_model_versions(
    max_results=MAX_RESULTS, order_by=["creation_timestamp DESC"]
)
versions = []
for version in versions_:
    version = dict(version)
    version.pop("aliases")
    versions.append(version)
versions = (
    pl.DataFrame(versions)
    .with_columns(
        pl.from_epoch("creation_timestamp", time_unit="ms"),
        pl.from_epoch("last_updated_timestamp", time_unit="ms"),
    )
    .drop("tags")
)
versions

creation_timestamp,current_stage,description,last_updated_timestamp,name,run_id,run_link,source,status,status_message,user_id,version
datetime[ms],str,str,datetime[ms],str,str,str,str,str,str,str,str
2024-12-09 21:25:59.981,"""None""","""""",2024-12-09 21:25:59.981,"""bikes""","""fc31f5b645204fdd9ef58fbbc0c105…","""""","""file:///home/fmind/mlops-pytho…","""READY""","""""","""""","""3"""
2024-12-09 20:15:47.252,"""None""","""""",2024-12-09 20:15:47.252,"""bikes""","""abad34d51a2a4cffa328136540b455…","""""","""file:///home/fmind/mlops-pytho…","""READY""","""""","""""","""2"""
2024-12-09 19:33:09.426,"""None""","""""",2024-12-09 19:33:09.426,"""bikes""","""ca4c7550ad354be9a2f10f7eade8d4…","""""","""file:///home/fmind/mlops-pytho…","""READY""","""""","""""","""1"""


# DASHBOARDS

In [9]:
px.strip(
    experiments,
    x="creation_time",
    color="lifecycle_stage",
    hover_name="name",
    hover_data=experiments.columns,
    title="Experiment Creation Time",
)

In [10]:
px.strip(
    models,
    x="creation_timestamp",
    hover_name="name",
    hover_data=models.columns,
    title="Model Creation Timestamp",
)

In [11]:
px.strip(
    models,
    x="creation_timestamp",
    hover_name="name",
    hover_data=models.columns,
    title="Model Creation Timestamp",
)

In [12]:
px.strip(
    versions,
    x="creation_timestamp",
    color="name",
    hover_name="name",
    hover_data=versions.columns,
    title="Version Creation Timestamp",
)

In [13]:
px.strip(
    runs,
    x="start_time",
    color="experiment_id",
    hover_name="run_name",
    hover_data=runs.columns,
    title="Run Start Time",
)

In [14]:
px.strip(
    runs,
    x="run_time",
    color="run_name",
    hover_name="run_id",
    hover_data=runs.columns,
    title="Run Time",
)